# Predict Personality Type from Chinese Text

Zihuan Ran

az_ran@outlook.com

In [53]:
import pkg_resources
import types
def get_imports():
    for name, val in globals().items():
        if isinstance(val, types.ModuleType):
            # Split ensures you get root package, 
            # not just imported function
            name = val.__name__.split(".")[0]

        elif isinstance(val, type):
            name = val.__module__.split(".")[0]

        # Some packages are weird and have different
        # imported names vs. system/pip names. Unfortunately,
        # there is no systematic way to get pip names from
        # a package's imported name. You'll have to add
        # exceptions to this list manually!
        poorly_named_packages = {
            "PIL": "Pillow",
            "sklearn": "scikit-learn"
        }
        if name in poorly_named_packages.keys():
            name = poorly_named_packages[name]

        yield name
imports = list(set(get_imports()))

# The only way I found to get the version of the root package
# from only the name of the package is to cross-check the names 
# of installed packages vs. imported packages
requirements = []
for m in pkg_resources.working_set:
    if m.project_name in imports and m.project_name!="pip":
        requirements.append((m.project_name, m.version))

for r in requirements:
    print("{}=={}".format(*r))

scipy==1.1.0
scikit-learn==0.22.2.post1
pandas==1.0.0
numpy==1.17.2
jieba==0.42.1


In this toy project, we aimed at predicting Yes/No for each of the five personalities given a piece of Chinese text:

In [1]:
import pandas as pd
import numpy as np
import jieba
from sklearn.feature_extraction.text import CountVectorizer

## Data Wrangling

In [2]:
label_data = pd.read_csv('data/essay_cleaned_zh_score.txt', sep='\t', header=None)

label_data.columns = ['id', 'o', 'c', 'e', 'a', 'n']

text_data = pd.read_csv('data/essay_cleaned_zh.txt', header=None, sep='\t')

text_data.columns = ['id', 'text']

In [3]:
text_data

,id,text
0,1,现在我刚从午睡中醒来。这有点奇怪，但自从我搬到得克萨斯州后，我一直无法集中精力处理事情。我记...
1,2,好吧，我们来看看意识流的文章。我以前在高中的时候经常做这样的事。他们很有趣，但我经常发现自己...
2,3,打开的键盘和按钮。事情终于成功了，我不需要使用句号逗号和所有那些想法。句点后加两倍空格。我们...
3,4,我真不敢相信！真的发生了！我的脉搏快疯了。所以这就是它的样子。现在我终于知道那是什么感觉了。...
4,5,好吧，我又来了一个好的旧意识流任务。我觉得我又回到了新生的英语课上了。这不是坏事，但是我的英...
...,...,...
2456,2463,我在家。我想上床睡觉，但我想起下周我有一个心理学作业要完成。也许这不会花那么长时间。我能应付...
2457,2464,一致性流。这个怎么拼？去他妈的如果我知道的话。我今天好像知道的不多。我他妈的为什么这么不高兴...
2458,2465,今天是12月8日星期三，这学期有很多事情要做。我正在努力尽可能地完成这学期的学习，但它并没有...
2459,2466,这个星期真是太可怕了。无论如何，现在是20分钟写作作业的时间了。我现在很累，今晚有很多功课要...


In [4]:
label_data

,id,o,c,e,a,n
0,1,1,0,0,1,1
1,2,1,0,0,0,0
2,3,0,1,0,1,1
3,4,1,1,1,0,0
4,5,1,0,1,0,1
...,...,...,...,...,...,...
2456,2463,0,1,0,1,0
2457,2464,0,0,1,1,1
2458,2465,1,0,0,0,0
2459,2466,0,0,0,1,1


In [5]:
full_data = text_data.join(label_data,lsuffix='_text', rsuffix='_label')

In [6]:
full_data

,id_text,text,id_label,o,c,e,a,n
0,1,现在我刚从午睡中醒来。这有点奇怪，但自从我搬到得克萨斯州后，我一直无法集中精力处理事情。我记...,1,1,0,0,1,1
1,2,好吧，我们来看看意识流的文章。我以前在高中的时候经常做这样的事。他们很有趣，但我经常发现自己...,2,1,0,0,0,0
2,3,打开的键盘和按钮。事情终于成功了，我不需要使用句号逗号和所有那些想法。句点后加两倍空格。我们...,3,0,1,0,1,1
3,4,我真不敢相信！真的发生了！我的脉搏快疯了。所以这就是它的样子。现在我终于知道那是什么感觉了。...,4,1,1,1,0,0
4,5,好吧，我又来了一个好的旧意识流任务。我觉得我又回到了新生的英语课上了。这不是坏事，但是我的英...,5,1,0,1,0,1
...,...,...,...,...,...,...,...,...
2456,2463,我在家。我想上床睡觉，但我想起下周我有一个心理学作业要完成。也许这不会花那么长时间。我能应付...,2463,0,1,0,1,0
2457,2464,一致性流。这个怎么拼？去他妈的如果我知道的话。我今天好像知道的不多。我他妈的为什么这么不高兴...,2464,0,0,1,1,1
2458,2465,今天是12月8日星期三，这学期有很多事情要做。我正在努力尽可能地完成这学期的学习，但它并没有...,2465,1,0,0,0,0
2459,2466,这个星期真是太可怕了。无论如何，现在是20分钟写作作业的时间了。我现在很累，今晚有很多功课要...,2466,0,0,0,1,1


In [ ]:
# 2460 <-> 2467, 6 rows skippped

In [7]:
full_data.shape

(2461, 8)

### Train test split

In [191]:
# set 85:15 train-test ratio
train = full_data.sample(frac=.85, random_state=101)

In [192]:
train.shape

(2092, 8)

In [193]:
test = full_data.drop(train.index)

In [194]:
test.shape

(369, 8)

In [44]:
# save for future reuse
train.to_csv('data/train.csv')
test.to_csv('data/test.csv')

## Models, Training and Evaluation

Goal: Given a piece of Chinese text, identify 1/0 for each of the five personalities.

Idea: 
    1. Five independent binary classification problem --> This approach is realized in this project
    2. Prediction on five altogether

In [51]:
# load split dataset
train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')

### Preprocess words data

The following 2 tools are used in this part for breaking text pragraph and vectorizing them:

* sklearn word vectorizer: [CountVectorizer](https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html)
* [jieba package for Chinese tokenization](https://github.com/fxsjy/jieba)

In [12]:
train['text'][0]

'所以我坐在我的房间里，想知道这是一个什么，我真的在想什么。我想到的日子似乎离我过去的希望相去甚远，我想你可以这么说。我在想未来的可能性，我在想不久前一位朋友曾经问过我，你的生活目标是什么。我告诉她我没有。今天我醒来，除了希望得到汤姆骨髓外，没有别的原因。这是一个谎言，我当然有目标，只是一个我现在正努力实现的目标。那我该怎么办？我想这就是我现在最想的。我也在窗外看到一幅美丽的风景，世界在阳光的照耀下滚滚而过，我不知道他们要讲什么故事。我需要一支雪茄。等等。上帝保佑尼古丁。这是一种很好的药。我的很多朋友都担心我会因此死去，但是嘿，我今天该怎么办？什么也没有。我可以把它修好，也许这就是我总是等着其他时间的坏时机的根源。我在考虑我该怎么办，星期五去，只是处于中度疼痛的状态，或者去做一些有趣的事情。天哪，我恨她的朋友，还有那些科学家和侄女。我真的不认为我能处理这些废话，我也不能处理马特试图成为每个人的朋友和关心的中心，老实说，我不知道该怎么办。我想对他大发雷霆，告诉他我对他的看法和他所有的废话，但我觉得这不适合我，反正还不适合。我要去买书。但我可能要等到明天。呵呵。不管怎样。所以我在一场大型歌剧的迷雾中，我没有让它从我身边经过，但我仍然在这里。怎么了。如果没有任何结果，那么无休止的乐观主义哲学又有什么意义呢。一大堆引语刚进入我的脑海，但我真的不想打字。好 啊。现在怎么办。我看到自己在我的生活中处于一个相当有问题的状态，没有去哪里，什么都不做，从来没有感到如此孤独，从来没有感觉如此活着。但话说回来。但总有一条路是正确的选择，但我看不到，没有明确的答案，至少不是我喜欢的。一个普通的kobi oshi maro不赢sinerio。好吧，废话！怎么办。我在这一点上几乎没有选择去不去等着看离开忘了付出牺牲报仇。怎么办？好吧，总有另一个选择，少走一条路。出去吧。我只是看不到，我需要看到它需要解放思想的欲望和我自己的需要分离和英格尔。怎么办？好吧，最好的例子，西内里奥，我什么都不做，只是随波逐流，很可能我会大便。我做了一些事情，事情只会变得更糟。我走了，让大家都很生气。付出我伤害索取我伤害别人忘记。如果我能忘记，我想我会忘记的，但你知道那对我来说永远都不管用。我可以隐藏，我可以等待，但忘记放手。从来都不是我擅长的。我已经得到了很多建议，所有这些都是一样的观望。但这对我来说不起作用，尽管我

In [3]:
l = jieba.cut(train['text'][0], cut_all=False)
' '.join(l)

Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/sy/r3wnpbxn4k355gs8m6fxrkfc0000gn/T/jieba.cache
Loading model cost 0.613 seconds.
Prefix dict has been built successfully.


'所以 我 坐在 我 的 房间 里 ， 想 知道 这是 一个 什么 ， 我 真的 在 想 什么 。 我 想到 的 日子 似乎 离 我 过去 的 希望 相去甚远 ， 我 想 你 可以 这么 说 。 我 在 想 未来 的 可能性 ， 我 在 想 不久前 一位 朋友 曾经 问过 我 ， 你 的 生活 目标 是 什么 。 我 告诉 她 我 没有 。 今天 我 醒来 ， 除了 希望 得到 汤姆 骨髓 外 ， 没有 别的 原因 。 这是 一个 谎言 ， 我 当然 有 目标 ， 只是 一个 我 现在 正 努力实现 的 目标 。 那 我 该 怎么办 ？ 我 想 这 就是 我 现在 最想 的 。 我 也 在 窗外 看到 一幅 美丽 的 风景 ， 世界 在 阳光 的 照耀 下 滚滚 而 过 ， 我 不 知道 他们 要 讲 什么 故事 。 我 需要 一支 雪茄 。 等等 。 上帝保佑 尼古丁 。 这是 一种 很 好 的 药 。 我 的 很多 朋友 都 担心 我会 因此 死去 ， 但是 嘿 ， 我 今天 该 怎么办 ？ 什么 也 没有 。 我 可以 把 它 修好 ， 也许 这 就是 我 总是 等 着 其他 时间 的 坏 时机 的 根源 。 我 在 考虑 我该 怎么办 ， 星期五 去 ， 只是 处于 中度 疼痛 的 状态 ， 或者 去 做 一些 有趣 的 事情 。 天 哪 ， 我 恨 她 的 朋友 ， 还有 那些 科学家 和 侄女 。 我 真的 不 认为 我能 处理 这些 废话 ， 我 也 不能 处理 马特 试图 成为 每个 人 的 朋友 和 关心 的 中心 ， 老实 说 ， 我 不 知道 该 怎么办 。 我 想 对 他 大发雷霆 ， 告诉 他 我 对 他 的 看法 和 他 所有 的 废话 ， 但 我 觉得 这 不 适合 我 ， 反正 还 不 适合 。 我要 去 买 书 。 但 我 可能 要 等到 明天 。 呵呵 。 不管怎样 。 所以 我 在 一场 大型 歌剧 的 迷雾 中 ， 我 没有 让 它 从 我 身边 经过 ， 但 我 仍然 在 这里 。 怎么 了 。 如果 没有 任何 结果 ， 那么 无休止 的 乐观主义 哲学 又 有 什么 意义 呢 。 一大堆 引语 刚 进入 我 的 脑海 ， 但 我 真的 不想 打字 。 好   啊 。 现在 怎么办 。 我 看到 自己 在 我 的 生活 中

In [4]:
vectorizer = CountVectorizer(analyzer='word', tokenizer=lambda x: jieba.cut(x,cut_all=False))
train_X = vectorizer.fit_transform(train['text'])#.apply(lambda x: ' '.join(jieba.cut(x,cut_all=False))))
test_X = vectorizer.transform(test['text'])#.apply(lambda x: ' '.join(jieba.cut(x,cut_all=False))))

In [5]:
len(vectorizer.get_feature_names())

29597

In [6]:
train_X.toarray().shape

(2092, 29597)

### Model1: SVM

In [7]:
from sklearn import svm

In [37]:
svm_o = svm.SVC(kernel='rbf')
svm_o.fit(train_X, train['o'])
svm_o.score(test_X, test['o'])

0.5203252032520326

In [9]:
svm_c = svm.SVC(kernel='rbf')
svm_c.fit(train_X, train['c'])
svm_c.score(test_X, test['c'])

0.5609756097560976

In [10]:
svm_e = svm.SVC(kernel='linear')
svm_e.fit(train_X, train['e'])

svm_e.score(test_X, test['e'])

0.5392953929539296

In [11]:
svm_a = svm.SVC(kernel='rbf')
svm_a.fit(train_X, train['a'])
svm_a.score(test_X, test['a'])

0.5555555555555556

In [12]:
svm_n = svm.SVC(kernel='rbf')
svm_n.fit(train_X, train['n'])
svm_n.score(test_X, test['n'])

0.6124661246612466

### Model2: Decision Tree

In [13]:
from sklearn import tree

In [14]:
dt_o = tree.DecisionTreeClassifier()
dt_o.fit(train_X, train['o'])
dt_o.score(test_X, test['o'])

0.5230352303523035

In [15]:
dt_c = tree.DecisionTreeClassifier()
dt_c.fit(train_X, train['c'])
dt_c.score(test_X, test['c'])

0.5149051490514905

In [16]:
dt_e = tree.DecisionTreeClassifier()
dt_e.fit(train_X, train['e'])
dt_e.score(test_X, test['e'])

0.5284552845528455

In [17]:
dt_a = tree.DecisionTreeClassifier()
dt_a.fit(train_X, train['a'])
dt_a.score(test_X, test['a'])

0.5013550135501355

In [18]:
dt_n = tree.DecisionTreeClassifier()
dt_n.fit(train_X, train['n'])
dt_n.score(test_X, test['n'])

0.5691056910569106

### Model3: Naive Bayes

In [19]:
from sklearn.naive_bayes import GaussianNB

In [20]:
nb_o = GaussianNB()
nb_o.fit(train_X.toarray(), train['o'])
nb_o.score(test_X.toarray(), test['o'])

0.5040650406504065

In [21]:
nb_c = GaussianNB()
nb_c.fit(train_X.toarray(), train['c'])
nb_c.score(test_X.toarray(), test['c'])

0.5284552845528455

In [22]:
nb_e = GaussianNB()
nb_e.fit(train_X.toarray(), train['e'])
nb_e.score(test_X.toarray(), test['e'])

0.5149051490514905

In [23]:
nb_a = GaussianNB()
nb_a.fit(train_X.toarray(), train['a'])
nb_a.score(test_X.toarray(), test['a'])

0.5040650406504065

In [24]:
nb_n = GaussianNB()
nb_n.fit(train_X.toarray(), train['n'])
nb_n.score(test_X.toarray(), test['n'])

0.5067750677506775

### Model4: Logistic Regression

In [25]:
from sklearn.linear_model import LogisticRegression

In [26]:
logi_o = LogisticRegression(solver="newton-cg")
logi_o.fit(train_X, train['o'])
logi_o.score(test_X, test['o'])

0.5230352303523035

In [27]:
logi_c = LogisticRegression(solver="newton-cg")
logi_c.fit(train_X, train['c'])
logi_c.score(test_X, test['c'])


0.5257452574525745

In [28]:
logi_e = LogisticRegression(solver="newton-cg")
logi_e.fit(train_X, train['e'])
logi_e.score(test_X, test['e'])


0.5447154471544715

In [29]:
logi_a = LogisticRegression(solver="newton-cg")
logi_a.fit(train_X, train['a'])
logi_a.score(test_X, test['a'])


0.5094850948509485

In [30]:
logi_n = LogisticRegression(solver="newton-cg")
logi_n.fit(train_X, train['n'])
logi_n.score(test_X, test['n'])

0.5691056910569106

###  Model5: Random Forest

In [31]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
rf_o = RandomForestClassifier(n_estimators=100)
rf_o.fit(train_X, train['o'])
rf_o.score(test_X, test['o'])

0.5311653116531165

In [33]:
rf_c = RandomForestClassifier(n_estimators=100)
rf_c.fit(train_X, train['c'])
rf_c.score(test_X, test['c'])

0.5582655826558266

In [34]:
rf_e = RandomForestClassifier(n_estimators=100)
rf_e.fit(train_X, train['e'])
rf_e.score(test_X, test['e'])

0.5582655826558266

In [35]:
rf_a = RandomForestClassifier(n_estimators=100)
rf_a.fit(train_X, train['a'])
rf_a.score(test_X, test['a'])

0.5420054200542005

In [36]:
rf_n = RandomForestClassifier(n_estimators=100)
rf_n.fit(train_X, train['n'])
rf_n.score(test_X, test['n'])

0.5663956639566395

## Closing and future outlooks

### Results

In this toy project, we aimed at predicting Yes/No for each of the five personalities given a piece of Chinese text. 

The training data we used are of size 2092 and testing data of 369 cases. Models tried and their respective result on each of 5 personalities are: 

|Model | O | C | E | A | N |
|:----:|:-:|:-:|:-:|:--:|:-:|
|SVM|0.52|0.56|0.54|0.56|0.61|
|Decision Tree|0.52|0.51|0.53|0.50|0.57|
|Naive Bayes|0.50|0.53|0.51|0.50|0.51|
|Logistic Regression|0.52|0.53|0.54|0.51|0.57|
|Random Forest|0.53|0.56|0.56|0.54|0.57|

Although prediction accuracy on testing data are just above 0.5 with best accuracy at 0.61(SVM for E: extraversion), a simple t-test can tell us the model is improving compared with baseline(p-value for SVM,Logistic regression and RF are 0.02, 0.03, 0.002 respectively):

In [38]:
from scipy import stats

In [47]:
stats.ttest_1samp([0.52, 0.56, 0.54, 0.56, 0.61], 0.5)

Ttest_1sampResult(statistic=3.8752880077301586, pvalue=0.017912803701679535)

In [48]:
stats.ttest_1samp([0.52, 0.53, 0.54, 0.51, 0.57], 0.5)

Ttest_1sampResult(statistic=3.3023719320147045, pvalue=0.029866968372355297)

In [49]:
stats.ttest_1samp([0.53, 0.56, 0.56, 0.54, 0.57], 0.5)

Ttest_1sampResult(statistic=7.0763037013736385, pvalue=0.0021047915678584256)

### Potential Improvements

Based on the conclusion above, this is shown to be a promsing path despite some barriers. Here are some ways that worth doing for accuracy improvement and future production:
1. More data: 
    we are training with only 2000+ text, each individual has string of length about 200 characters. In crease in either of the two dimension would improve performace of the models.
    
2. More evaluation metrics:
    As a binary classification problem, there are many other diagonisis metrics that may enlighten us on which aspects to improve. Some of them are precision, recall, F1-score, and ROC curve once more data avaliable.
    
3. Finer tuned model:
    In this project, a wide try of 5 models are produced roughly for initial reference. One next step may be to better tune those who seems to have potential: SVM and Random forest for example.
    
4. Improved preprocessing:
    Precprocessing text is a crucial step and one possible improvement is to create a keyword list of each of 5 personalities and take them to weight segmented text...

Thank you for your time looking at this project. Please let me know at az_ran@outlook.com if there are any questions. Cheers!